<a href="https://colab.research.google.com/github/Vibha1v/Praja-Seva-Srujana-hackathon/blob/main/MARK_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
#installing all the libs, keep on adding to this when you need more and more libs at runtime
!pip install langchain
!pip install langchain-community
!pip install ctransformers
!pip install sentence-transformers
!pip install chromadb
!pip install pandas
!pip install huggingface-hub


In [8]:
# 2nd cell
import os
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_community.llms import CTransformers
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from huggingface_hub import hf_hub_download
import pandas as pd
from langchain_community.document_loaders import TextLoader
import zipfile

In [9]:
# --- Prepare the Medication Knowledge Base from the text files ---
medication_kb_path = "/content/medication_kb"
medication_docs = []
if os.path.exists(medication_kb_path):
    for filename in os.listdir(medication_kb_path):
        if filename.endswith(".txt"):
            loader = TextLoader(os.path.join(medication_kb_path, filename))
            medication_docs.extend(loader.load())

if not medication_docs:
    print("Error: No medication files found. Please ensure your medication files are uploaded to the /content/medication_kb folder.")
    exit()

medication_text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
medication_split_docs = medication_text_splitter.split_documents(medication_docs)
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
medication_vector_store = Chroma.from_documents(medication_split_docs, embeddings)
print("Medication Knowledge Base created from text files.")
print("-" * 50)


# --- Prepare the Symptom Alternatives from the provided files ---
symptom_kb_path = "/content/symptoms_kb"
symptom_docs = []
if os.path.exists(symptom_kb_path):
    for filename in os.listdir(symptom_kb_path):
        if filename.endswith(".txt"):
            loader = TextLoader(os.path.join(symptom_kb_path, filename))
            symptom_docs.extend(loader.load())

if not symptom_docs:
    print("Error: Symptom alternatives files not found. Please upload your files to the /content/symptoms_kb directory.")
    exit()

symptom_text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100, separators=["*"])
symptom_split_docs = symptom_text_splitter.split_documents(symptom_docs)
symptom_vector_store = Chroma.from_documents(symptom_split_docs, embeddings)
print("Symptoms Knowledge Base created from provided text files.")
print("-" * 50)

Medication Knowledge Base created from text files.
--------------------------------------------------
Symptoms Knowledge Base created from provided text files.
--------------------------------------------------


In [10]:
# --- 3. Download and Load the Local LLM (UPGRADED FOR COLAB PRO) ---
# Switched to a larger, more powerful model: Mistral 7B Instruct v0.2
model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
# Using a high-quality 5-bit quantized version for better accuracy
model_file = "mistral-7b-instruct-v0.2.Q5_K_M.gguf"

model_path = hf_hub_download(repo_id=model_name, filename=model_file)

llm = CTransformers(
    model=model_path,
    # Updated model_type back to "mistral"
    model_type="mistral",
    # Increased context_length to take advantage of the better model
    config={'max_new_tokens': 256, 'temperature': 0.1, 'context_length': 4096},
    # ESSENTIAL FOR GPU: Offloads model layers to the GPU for fast performance.
    gpu_layers=50
)
print("Upgraded model loaded successfully on GPU.")
print("-" * 50)

Upgraded model loaded successfully on GPU.
--------------------------------------------------


In [16]:
# --- 4. Interactive Command-Line Interface (Two-Step RAG Logic) ---
print("Welcome to the Health AI Agent! Type 'exit' to quit.")
while True:
    medication_input = input("Pharmacist: Enter the medication name or a symptom. ")
    if medication_input.lower() == "exit":
        break

    try:
        is_medication_query = False
        is_symptom_query = False

        # First, check the medication knowledge base
        retriever_med = medication_vector_store.as_retriever(search_kwargs={'k': 3})
        med_info_docs = retriever_med.invoke(medication_input)

        if med_info_docs:
            # Check if the primary retrieved document is actually a medication
            # This is a robust way to check if the input truly matched a medication
            primary_doc_source = med_info_docs[0].metadata.get('source', '')
            if 'medication_kb' in primary_doc_source:
                is_medication_query = True

        # If not a medication, check the symptom knowledge base
        if not is_medication_query:
            retriever_symptom = symptom_vector_store.as_retriever()
            symptom_info_docs = retriever_symptom.invoke(medication_input)
            if symptom_info_docs:
                primary_doc_source = symptom_info_docs[0].metadata.get('source', '')
                if 'symptoms_kb' in primary_doc_source:
                    is_symptom_query = True

        if is_medication_query:
            # Handle Medication Query
            med_info = med_info_docs[0].page_content
            source_med_name = med_info_docs[0].metadata.get('source', 'Unknown')
            print(f"\nAgent: Found information for '{source_med_name}'.")

            # This line is correct for medication documents
            symptom_query = med_info.split('Uses: ')[1].split('\n')[0].strip().split(',')[0].strip()
            print(f"Agent: Identifying the primary use as: '{symptom_query}'.")

            alt_info_docs = retriever_symptom.invoke(symptom_query)
            if not alt_info_docs:
                print(f"Agent: I could not find alternative medications for '{symptom_query}'.")
                continue
            alt_info = alt_info_docs[0].page_content

            final_query = f"""
            You are a helpful AI assistant for pharmacists. Your task is to provide clear, safe, and concise information based on the provided context. Do not hallucinate or make up information.
            ---
            The pharmacist has asked for an alternative to the medication: {medication_input}.
            Here is the information I have for the primary medication:
            {med_info}
            Here is a list of other medications with similar uses:
            {alt_info}
            ---
            Based ONLY on the information above, please provide a response with the following structure:
            1. Summary of the Medication:
            - What is the medication?
            - What is its primary use?
            2. Suggested Alternatives:
            - Provide a bulleted list of the alternative medications and remedies found in the context.
            3. Safety Disclaimer:
            - Add a final, prominent warning that this information is for reference only and not a substitute for professional medical advice. Always consult a qualified healthcare professional before making any decisions.
            """
            response = llm.invoke(final_query)
            print(f"\nAgent: {response}\n")

        elif is_symptom_query:
            # Handle Symptom Query
            symptom_info = symptom_info_docs[0].page_content
            source_symptom_name = symptom_info_docs[0].metadata.get('source', 'Unknown')
            print(f"\nAgent: Found information for '{source_symptom_name}'.")

            final_query = f"""
            You are a helpful AI assistant for pharmacists. Your task is to provide a list of alternative medications and home remedies for the following symptom. Do not hallucinate or make up information.

            ---

            The pharmacist has asked for remedies for the symptom: {medication_input}.

            Here is the information I have for this symptom:
            {symptom_info}

            ---

            Based ONLY on the information above, please provide a response with the following structure:

            1. Symptom Summary:
            - What is the symptom?

            2. Suggested Alternatives:
            - Provide a bulleted list of the alternative medications found in the context.

            3. Lifestyle and Home Remedies:
            - Provide a bulleted list of the lifestyle and home remedies found in the context.

            4. Safety Disclaimer:
            - Add a final, prominent warning that this data is synthetic and not a substitute for professional medical advice. Always consult a qualified healthcare professional before making any decisions.
            """

            # The LLM's response is a single string.
            # We can format it here before printing.
            response_text = llm.invoke(final_query)

            # Split the response into sections based on the markdown headers
            # to format them neatly.
            sections = response_text.split('4. Safety Disclaimer:')

            if len(sections) > 1:
                main_body = sections[0].strip()
                disclaimer = "4. Safety Disclaimer:" + sections[1].strip()

                # Replace markdown asterisks for cleaner display
                main_body = main_body.replace('*1.', '1.')
                main_body = main_body.replace('*2.', '2.')
                main_body = main_body.replace('*3.', '3.')
                main_body = main_body.replace('*', '').strip()

                # Print the formatted output
                print("\n" + main_body)
                print("\n" + disclaimer + "\n")
            else:
                # If the split fails, just print the raw response
                print(f"\nAgent: {response_text}\n")

    except KeyboardInterrupt:
        print("\nProgram terminated by user.")
        break
    except Exception as e:
        print(f"An error occurred: {e}")
        continue

print("Thank you for using the Health AI Agent. Have a great day!")

Welcome to the Health AI Agent! Type 'exit' to quit.
Pharmacist: Enter the medication name or a symptom. crocin

Agent: Found information for '/content/medication_kb/crocin.txt'.
Agent: Identifying the primary use as: 'Fever'.

Agent: 1. Summary of Medication:
            The medication Crocin is used primarily for the treatment of fever and headaches.
            2. Suggested Alternatives:
            - Ibuprofen: A nonsteroidal anti-inflammatory drug (NSAID) used to reduce pain, inflammation, and fever.
            - Acetaminophen: A pain reliever and fever reducer. It does not have anti-inflammatory properties.
            - Naproxen: Another NSAID used for the relief of pain, inflammation, and fever.
            - Lukewarm sponge bath: A method to help reduce body temperature in cases of fever.
            - Hydration: Drinking plenty of fluids to prevent dehydration caused by sweating during a fever.
            - Rest: Allowing the body to focus its energy on fighting the infecti